In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

# Face detect and cropping

In [5]:
!pip install mtcnn

In [6]:
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

0.1.0


In [7]:
# face detection for the 5 Celebrity Faces Dataset
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
import numpy as np
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
import cv2
import dlib

In [8]:
## extracting face under one folder alone
#warnings.filterwarnings("ignore") 

# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
    
    image = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)
    #using MTCNN for detecting features
    detector = MTCNN()
    results = (detector.detect_faces(image))
    print(results)

    if not results:
      
          print("no result")
          return None
    else: 

        bounding_box = results[0]['box']
        keypoints = results[0]['keypoints']
        r = dlib.rectangle(bounding_box[0], bounding_box[1], bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3])     

        crop_img = image[bounding_box[1]:bounding_box[1]+ bounding_box[3], bounding_box[0]:bounding_box[0]+bounding_box[2]]
        crop_img1 = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
        scale_percent = 220 # percent of original size
        width = int(crop_img1.shape[1] * scale_percent / 100)
        height = int(crop_img1.shape[0] * scale_percent / 100)
        dim = (width, height)
        # resize image
        resized = cv2.resize(crop_img1, dim, interpolation = cv2.INTER_AREA)
        reimage = cv2.resize(resized, (160,160), interpolation = cv2.INTER_AREA)
        face_array = asarray(reimage)      
        return face_array




In [9]:
# #FOR DEBUG ONLY
# # specify folder to plot
# folder = '/content/drive/MyDrive/FYP DY2/5 celebrities dataset/train_masked/Kishore_Kandasamy/'
# i = 1

# for filename in listdir(folder):
#     path = folder + filename
#     print(path)
#     face = extract_face(path)
  
#     if face is not None:
#       print(i, face.shape)
#       pyplot.subplot(2, 10, i)
#       pyplot.axis('off')
#       #face1 = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
#       pyplot.imshow(face)
#     i += 1

# New section

In [10]:
# load images and extract faces for all images in a directory
def load_faces(directory):
	faces = list()
	# enumerate files
	for filename in listdir(directory):
		# path
		path = directory + filename
		# get face
		face = extract_face(path)
		# store
		if face is not None:
			faces.append(face)
	return faces
 
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
	X, y = list(), list()
	# enumerate folders, on per class
	for subdir in listdir(directory):
		# path
		path = directory + subdir + '/'
		# skip any files that might be in the dir
		if not isdir(path):
			continue
		# load all faces in the subdirectory
		faces = load_faces(path)
		# create labels
		labels = [subdir for _ in range(len(faces))]
		# summarize progress
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		# store
		X.extend(faces)
		y.extend(labels)
	return asarray(X), asarray(y)
 
def load_save_dataset():
  # load MASKED train dataset
    trainX, trainy = load_dataset('C:/Users/Tanya/Desktop/FYP/FYP - Full Stack/5 celebrities dataset/train_masked/')
  # print(trainX.shape, trainy.shape)
  # load test dataset
    testX, testy = load_dataset('C:/Users/Tanya/Desktop/FYP/FYP - Full Stack/5 celebrities dataset/val_masked/')
  # save arrays to one file in compressed format
  # print(trainX,trainy)
    print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
    savez_compressed('C:/Users/Tanya/Desktop/FYP/FYP - Full Stack/5-celebrity-faces-dataset_masked.npz', trainX, trainy, testX, testy)

    data = np.load('C:/Users/Tanya/Desktop/FYP/FYP - Full Stack/5-celebrity-faces-dataset_masked.npz')

    trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
    print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

load_save_dataset()


[{'box': [868, 930, 1824, 2043], 'confidence': 0.9996775388717651, 'keypoints': {'left_eye': (1592, 1699), 'right_eye': (2248, 1675), 'nose': (1979, 2145), 'mouth_left': (1611, 2372), 'mouth_right': (2196, 2386)}}]
[{'box': [908, 1545, 1614, 1813], 'confidence': 0.9985870122909546, 'keypoints': {'left_eye': (1527, 2147), 'right_eye': (2149, 2126), 'nose': (1869, 2536), 'mouth_left': (1521, 2772), 'mouth_right': (2067, 2792)}}]
[{'box': [889, 873, 1818, 2165], 'confidence': 0.9988358616828918, 'keypoints': {'left_eye': (1380, 1703), 'right_eye': (2217, 1660), 'nose': (1800, 2093), 'mouth_left': (1476, 2541), 'mouth_right': (2080, 2522)}}, {'box': [91, 3454, 217, 280], 'confidence': 0.8314775824546814, 'keypoints': {'left_eye': (148, 3570), 'right_eye': (198, 3556), 'nose': (148, 3592), 'mouth_left': (139, 3655), 'mouth_right': (172, 3645)}}]
>loaded 3 examples for class: Kishore_Kandasamy
[{'box': [468, 563, 1232, 1624], 'confidence': 0.9989217519760132, 'keypoints': {'left_eye': (827, 

[{'box': [460, 388, 379, 435], 'confidence': 0.9999959468841553, 'keypoints': {'left_eye': (556, 538), 'right_eye': (734, 527), 'nose': (631, 616), 'mouth_left': (572, 715), 'mouth_right': (711, 708)}}]
[{'box': [371, 290, 445, 519], 'confidence': 0.9999115467071533, 'keypoints': {'left_eye': (479, 494), 'right_eye': (667, 467), 'nose': (564, 603), 'mouth_left': (512, 691), 'mouth_right': (666, 669)}}]
>loaded 4 examples for class: Pranjal_Mathur
[{'box': [1197, 1349, 1263, 1621], 'confidence': 0.999852180480957, 'keypoints': {'left_eye': (1521, 1962), 'right_eye': (2083, 1979), 'nose': (1781, 2328), 'mouth_left': (1573, 2611), 'mouth_right': (2040, 2628)}}]
[{'box': [1138, 1425, 1059, 1250], 'confidence': 0.9993190765380859, 'keypoints': {'left_eye': (1472, 1888), 'right_eye': (1883, 1903), 'nose': (1660, 2090), 'mouth_left': (1480, 2260), 'mouth_right': (1866, 2266)}}, {'box': [2838, 4002, 27, 32], 'confidence': 0.8584333658218384, 'keypoints': {'left_eye': (2850, 4014), 'right_eye':

[{'box': [560, 375, 1248, 1680], 'confidence': 0.9974479675292969, 'keypoints': {'left_eye': (937, 1108), 'right_eye': (1488, 1114), 'nose': (1217, 1501), 'mouth_left': (967, 1707), 'mouth_right': (1427, 1707)}}]
>loaded 1 examples for class: Meghna_Gupta
[{'box': [429, 326, 455, 515], 'confidence': 0.9974459409713745, 'keypoints': {'left_eye': (553, 517), 'right_eye': (742, 508), 'nose': (635, 631), 'mouth_left': (561, 713), 'mouth_right': (721, 710)}}]
>loaded 1 examples for class: Pranjal_Mathur
[{'box': [1191, 1626, 1140, 1408], 'confidence': 0.9995244741439819, 'keypoints': {'left_eye': (1390, 2195), 'right_eye': (1872, 2152), 'nose': (1583, 2502), 'mouth_left': (1463, 2733), 'mouth_right': (1863, 2706)}}]
>loaded 1 examples for class: Sanjana_Kurkimat
Loaded:  (15, 160, 160, 3) (15,) (4, 160, 160, 3) (4,)
Loaded:  (15, 160, 160, 3) (15,) (4, 160, 160, 3) (4,)
